# Employment-Unemployment Model

<br>
<br>

**Goal**:

Estimate a Markov chain using employment data

In [43]:
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

## Steps to "understanding the world around us" (a short digression)


Our friend Jim Savage has thought about what a "modern statistical workflow" entails. We summarize some steps he has proposed for understanding the world around us:

> 1. Prepare and visualize your data.
> 2. Create a generative model for the data...
>   - A first model should be as simple as possible.
> 3. Simulate some artificial data from your model given some assumed parameters that you "pick out of a hat" ($\theta$)
> 4. Use the artificial data to estimate the model parameters ($\hat{\theta})$
> 5. Check that you recovered a good approximation of the "known unknowns" (aka, $\theta \approx \hat{\theta}$)
>   - Possibly repeat 3-5 with different estimators and true parameters ($\theta$), to get an understanding of how well the fitting procedure works
> 6. Fit the model to your real data, check the fit
> 7. Argue about the results with your friends and colleagues
> 8. Go back to 2. with a slightly richer model. Repeat.
> 9. Think carefully about what decisions will be made from the analysis, encode a loss function, and perform statistical decision analysis... Note that in many cases, we will do step 9 before steps 1-8!

Later this semester, we will talk formally about what it means to "fit" your model (and the work that it entails), but, for now, we find it sufficient to say that it's a process to ensure that the probability distribution over outcomes generated by your model lines up with the data (aka, finding the right model parameters).

We'll do a version of steps 1-6 to help us improve our understanding of the labor data that we previously saw.

### Our plan

1. Prepare and visualize our data.
2. Develop a generative model of employment and unemployment
3. Simulate data from our generative model for given parameters
4. Fit our model to the simulated data
5. Explore different ways that we might have chosen to fit the data
6. Fit the model with the BLS data
7. Examine what our model implies for the effects of COVID on employment/unemployment

## Step 1: Prepare and visualize our data

We have done this in earlier lectures and will not repeat the work here!

## Step 2: Create a generative model

**A simple model of employment**

In the vein of, "the first model created should be as simple as possible", we use the employment model that we studied earlier.

Consider a single individual that transitions between employment and unemployment

* When unemployed, they find a new job with probability $\alpha$
* When employed, they lose their job with probability $\beta$

<br>
<br>

![ModelFlowchart](model_diagram.png)


## Step 3: Simulate data from our generative model

<br>
<br>

We will break simulating data from the model into two steps:

1. Given today's state and the transition probabilities, draw from tomorrow's state
2. Given an initial state and transition probabilities, simulate an entire history of employment/unemployment using the one-step transition kernel


**Simulate the one-step employment transition**

In [44]:
# alfa and beta are float type here

In [45]:
def next_state(s_t, alpha, beta):
    """
    Transitions from employment/unemployment in period t to
    employment/unemployment in period t+1
    
    Parameters
    ----------
    s_t : int
        The individual's current state... s_t = 0 maps to
        unemployed and s_t = 1 maps to employed
    alpha : float
        The probability that an individual goes from
        unemployed to employed
    beta : float
        The probability that an individual goes from
        employed to unemployed

    Returns
    -------
    s_tp1 : int
        The individual's employment state in `t+1`
    """
    # follow good coding habits and write down the documentation

    # Draw a random number
    u_t = np.random.rand() # 0=unemployed, 1=employed

    # Let 0 be unemployed... If unemployed and draws --> s_t = 0
    # a value less than lambda then becomes employed --> when is she going to be employed again
    if (s_t == 0) and (u_t < alpha):
        s_tp1 = 1
    # Let 1 be employed... If employed and draws a
    # value less than beta then becomes unemployed
    elif (s_t == 1) and (u_t < beta):
        s_tp1 = 0
    # Otherwise, he keeps the same state as he had
    # at period t
    else:
        s_tp1 = s_t

    return s_tp1

    # this function depends on the markov property


Notice how this function incorporates the Markov property that our model assumes.

The Markov property says $\text{Probability}(s_{t+1} | s_{t}) = \text{Probability}(s_{t+1} | s_{t}, s_{t-1}, \dots, s_0)$.

This means that, other than the transition probabilities, we only need to know today's state and not the entire history.

**Testing our function**

It's always a good idea to write some simple test cases for functions that we create.

In [46]:
next_state(0, 0.5, 0.5)

# The individual's current state... s_t = 0 maps to unemployed and s_t = 1 maps to employed
# The probability that an individual goes from unemployed to employed
#        The probability that an individual goes from employed to unemployed

1

In [47]:
# Should never become employed from unemployment
# if alpha is 0
next_state(0, 0.0, 0.5) == 0

# she should stay unemployed

True

In [48]:
# Should never become unemployed from employment
# if beta is 0
next_state(1, 0.5, 0.0) == 1

# should stay employed

True

In [49]:
# Should always transition to employment from unemployment
# when alpha is 1
next_state(0, 1.0, 0.5) == 1

True

In [50]:
# Should always transition to unemployment from employment
# when beta is 1
next_state(1, 0.5, 1.0) == 0

True

**Simulate entire history**

**Note**: Later we will allow $\alpha$ and $\beta$ to change over time, so while we want you think of them as constant for now, we will write code that allows for them to fluctate period-by-period

In [51]:
# now we want alfa and beta to fluctuate period by period
# alfa and beta are numpy array type of data

In [52]:
def simulate_employment_history(alpha, beta, s_0):
    """
    Simulates the history of employment/unemployment. It
    will simulate as many periods as elements in `alpha`
    and `beta`
    
    Parameters
    ----------
    alpha : np.array(float, ndim=1)
        The probability that an individual goes from
        unemployed to employed
    beta : np.array(float, ndim=1)
        The probability that an individual goes from
        employed to unemployed
    s_0 : int
        The initial state of unemployment/employment, which
        should take value of 0 (unemployed) or 1 (employed)
    """
    # Create array to hold the values of our simulation
    # we got to make sure that alfa and beta are from the same length
    
    assert(len(alpha) == len(beta))
    T = len(alpha)
    s_hist = np.zeros(T+1, dtype=int)

    s_hist[0] = s_0
    for t in range(T):
        # Step one period into the future
        s_0 = next_state(s_0, alpha[t], beta[t])  # Notice alpha[t] and beta[t]
        s_hist[t+1] = s_0

    return s_hist

**Check output of the function**

In [53]:
# in order to use the function, we need to know whether the person was employed or unemployed
# set first value of the function with the previous state

alpha = np.ones(50)*0.25
beta = np.ones(50)*0.025

# probability of losing a job can is lower than the prob of finding a job

simulate_employment_history(alpha, beta, 0)

# i can put 0 or 1 in the simulate function to see what happens
# thus it doesnt take long for ther to find a job ([period 2])
# thus our simulated method has succeded

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1])

## Step 4: Fit your model to our artificial data

There are lots of procedures that one could use to infer parameters values from data. Here we'll just count relative frequencies of transitions.

Let's think about the general case. Consider an $N$-state Markov chain. The parameters of the Markov chain are the elements of the transition matrix, $P$.

$$P \equiv \begin{bmatrix} p_{11} & p_{12} & \dots & p_{1N} \\ p_{21} & \vdots & \ddots & \vdots \\ p_{N1} & p_{N2} & \dots & p_{NN} \end{bmatrix}$$

Let $\{y_0, y_1, \dots, y_T\}$ be a sequence of observations generated from the $N$-state Markov chain, then our "fitting" procedure would assign the following value to $p_{ij}$:

$$p_{ij} = \frac{\sum_{t=0}^T \mathbb{1}_{y_{t} == i} \mathbb{1}_{y_{t+1} == j}}{\sum_{t=0}^T \mathbb{1}_{y_{t} == i}}$$

**Note**: If you'd like to understand why this procedure makes sense, we recommend computing $\sum_{j=1}^N p_{ij}$ for a given $i$. What value do you get? Why?

In [54]:
# Now we want to fit out simu;ated model to artifiocial data
# p11 means that agent that is in state 1 is going to be in state 1 next period and so on till Nth term
# p12 means that agent that is in state 1 is going to be in state 2 next period and so on till Nth term

# example from video
    # artificial data : 0111001
    # it takes 1 if the condition below is true, or zero otherwise
    # over the summation of the places where yt was equal to 0
    # then we have 1.0+1.1+0.1/1+1+1= 1/3
    # denoominator= shows the number of times where the sequence goes from 0 to 0 and 0 to 1
    # all times that we go from i to j

In [55]:
# now, lets write some code that can count frequences
# THE PROBABILITY OF TRANSISIOTING FROM STATE 0 TO 1 AND VICEVERSA
# all the possible states = idx--> 01234... T
# History is going to give me TRUE FALSE TRUE etc
# idx < T-1)--> TRUE TRUE TRUE... FALSE at T
# SEE THE NOW BELOW

**Counting frequencies**



In [56]:
def count_frequencies_individual(history):
    """
    Computes the transition probabilities for a two-state
    Markov chain--> # therefore we are going to specialize

    Parameters
    ----------
    history : np.array(int, ndim=1)
        An array with the state values of a two-state Markov chain
    
    Returns
    -------
    alpha : float
        The probability of transitioning from state 0 to 1
    beta : float
        The probability of transitioning from state 1 to 0
    """
    # Get length of the simulation and an index tracker
    T = len(history)
    idx = np.arange(T)


# this step is to find where the zeros and the ones are
    # Determine when the chain had values 0 and 1 -- Notice
    # that we can't use the last value because we don't see
    # where it transitions to
    zero_idxs = idx[(history == 0) & (idx < T-1)] # THIS IS GOING TO GIVE US SOME TRUES AND FALSES
    one_idxs = idx[(history == 1) & (idx < T-1)] # THIS IS GOING TO GIVE US SOME TRUES AND FALSES

# here im using the information of where the zeros and the ones are, to figure out what the corresponding counts of frequencies should be
    # Check what percent of the t+1 values were 0/1
    # HOW DOES THE COUNTING PROCEDUREE WORK
    alpha = np.sum(history[zero_idxs+1]) / len(zero_idxs) # NOW WE ARE GOING TO SUM UP THE HISTORY FOR ALPHA
    beta = np.sum(1 - history[one_idxs+1]) / len(one_idxs) # THEN WE DO IT 

    return alpha, beta

**Checking the fit**

In [57]:
# now we want to check the accuracy of fitting out model to a simulated model (the count of frequencies model)

In [58]:
def check_accuracy(T, alpha=0.25, beta=0.025):
    """
    Checks the accuracy of our fit by printing the true values
    and the fitted values for a given T
    
    Parameters
    ----------
    T : int
        The length of our simulation
    alpha : float
        The probability that an individual goes from
        unemployed to employed
    beta : float
        The probability that an individual goes from
        employed to unemployed
    """
    idx = np.arange(T)
    alpha_np = np.ones(T)*alpha
    beta_np = np.ones(T)*beta

    # Simulate a sample history
    emp_history = simulate_employment_history(alpha_np, beta_np, 0)

    # Check the fit
    alpha_hat, beta_hat = count_frequencies_individual(emp_history)
    

    # finally we peint out the true values and the fitted values
    print(f"True alpha was {alpha} and fitted value was {alpha_hat}")
    print(f"True beta was {beta} and fitted value was {beta_hat}")
    
    return alpha, alpha_hat, beta, beta_hat

In [59]:
check_accuracy(10_000, 0.25, 0.025)

# we find that we are getting correct results that are very similar to our initial parameters--> 0.25 and 0.025
# that is good. it means that we had enough data

True alpha was 0.25 and fitted value was 0.25843881856540085
True beta was 0.025 and fitted value was 0.02695536897923111


(0.25, 0.25843881856540085, 0.025, 0.02695536897923111)

Well... If we observe 10,000 months of employment history for someone then we know that we can back out the parameters of our models...

Unfortunately, our real world data won't have that much information.

In [60]:
# what if if we have less data than 10,000

What about for an entire lifetime of employment transitions?

In [61]:
check_accuracy(45*12, 0.25, 0.025)

# this is particularly inaccurate 0.316666666
# there could have been structural changes

True alpha was 0.25 and fitted value was 0.32558139534883723
True beta was 0.025 and fitted value was 0.026156941649899398


(0.25, 0.32558139534883723, 0.025, 0.026156941649899398)

What about for just two years of observations?

In [62]:
check_accuracy(2*12, 0.25, 0.025)

# two years of data, the fitting procedure can be very noisy

True alpha was 0.25 and fitted value was 1.0
True beta was 0.025 and fitted value was 0.045454545454545456


(0.25, 1.0, 0.025, 0.045454545454545456)

In [63]:
# WE COULD APPLY CROSS SECTIONAL ANALYSIS BUT WE WOULD NEED INDEPENDENCE

## Step 3 and 4 (second try)

Data for the employment history of a single individual will not give us a good chance of fitting our model accurately...

However, the BLS isn't infering EU/UE rates from its observation of a single individual. Rather, they're using a cross-section of individuals!

Can we use a cross-section rather than for one individual?

Yes, but, in order for a version of our "frequency counting" procedure to work, we want independence across individuals, i.e.

$$\text{Probability}(s_{i, t+1}, s_{j, t+1} | s_{i, t}, s_{j, t}, \alpha, \beta) = \text{Probability}(s_{i, t+1} | s_{i, t}, \alpha, \beta) \text{Probability}(s_{j, t+1} | s_{j, t}, \alpha, \beta)$$

When we observed only a single individual, the Markov property did a lot of the work to get independence for us.

When might this not be the case?

* Change in government policy results in a "jobs guarantee"
* Technological change results in the destruction of an entire industries jobs
* Recession causes increased firing across entire country

(Spoiler alert: Some of these will present problems for us... which is why we'll allow for $\alpha$ and $\beta$ to move each period)

(TODO: Tom said he'd like to edit this cell again)

In [64]:
# {Probability}(s_{i, t+1}, s_{j, t+1} | s_{i, t}, s_{j, t}, \alpha, \beta)

# THIS APPEARRS ABOVE
# joint distribution of states conditional on the iniitial joint distribution plus the initial states = products of their independent distributions

In [65]:
# now we want to simulate the cross section

**Simulating a cross-section**

In [66]:
def simulate_employment_cross_section(alpha, beta, s_0, N=500):
    """
    Simulates a cross-section of employment/unemployment using
    the model we've described above.
    
    Parameters
    ----------
    alpha : np.array(float, ndim=1)
        The probability that an individual goes from
        unemployed to employed
    beta : np.array(float, ndim=1)
        The probability that an individual goes from
        employed to unemployed
    s_0 : np.array(int, ndim=1)
        The fraction of the population that begins in each
        employment state
    N : int
        The number of individuals in our cross-section
    
    Returns
    -------
    s_hist_cs : np.array(int, ndim=2)
        An `N x T` matrix that contains an individual
        history of employment along each row
    """
    # we are going to store the different states in a big matrix


    # Make sure transitions are same size and get the length
    # of the simulation from the length of the transition
    # probabilities
    assert(len(alpha) == len(beta))
    T = len(alpha)

    # Check the fractions add to one and figure out how many
    # zeros we should have
    assert(np.abs(np.sum(s_0) - 1.0) < 1e-8)
    Nz = np.floor(s_0[0]*N).astype(int)

    # Allocate space to store the simulations
    s_hist_cs = np.zeros((N, T+1), dtype=int)
    s_hist_cs[Nz:, 0] = 1
    
    for i in range(N):
        s_hist_cs[i, :] = simulate_employment_history(
            alpha, beta, s_hist_cs[i, 0]
        )
    
    return s_hist_cs


In [67]:
alpha = np.ones(1)*0.25
beta = np.ones(1)*0.025

simulate_employment_cross_section(alpha, beta, np.array([0.35, 0.65]), 10)

array([[0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1]])

**Store the simulation in pandas**

Real world data will typically be stored in a DataFrame, so let's store our artificial data in a DataFrame as well

In [68]:
def pandas_employment_cross_section(eu_ue_df, s_0, N=500):
    """
    Simulate a cross-section of employment experiences
    
    Parameters
    ----------
    eu_ue_df : pd.DataFrame
        A DataFrame with columns `dt`, `alpha`, and `beta`
        that have the monthly eu/ue transition rates
    s_0 : np.array(float, ndim=1)
        The fraction of the population that begins in each
        employment state
    N : int
        The numbers of individuals in our cross-section

    Returns
    -------
    df : pd.DataFrame
        A DataFrame with the dates and an employment outcome
        associated with each date of `eu_ue_df`
    """
    # Make sure that `ue_ue_df` is sorted by date
    eu_ue_df = eu_ue_df.sort_values("dt")
    alpha = eu_ue_df["alpha"].to_numpy()
    beta = eu_ue_df["beta"].to_numpy()

    # we will extract our alpha and data arrays

    # Simulate cross-section
    employment_history = simulate_employment_cross_section(
        alpha, beta, s_0, N
    )

    df = pd.DataFrame(employment_history[:, :-1].T)
    df = pd.concat([eu_ue_df["dt"], df], axis=1)
    df = pd.melt(
        df, id_vars=["dt"],
        var_name="pid", value_name="employment"
    )

    return df

In [69]:
T = 6
eu_ue_df = pd.DataFrame(
    {
        "dt": pd.date_range("2018-01-01", periods=T, freq="MS"), 
        "alpha": np.ones(T)*0.25,
        "beta": np.ones(T)*0.025
    }
)

df = pandas_employment_cross_section(eu_ue_df, np.array([0.25, 0.75]), N=5_000)
df.head(12)

# we can see that the individual was unemployed in the first 6 months

,dt,pid,employment
0,2018-01-01,0,0
1,2018-02-01,0,0
2,2018-03-01,0,0
3,2018-04-01,0,0
4,2018-05-01,0,0
5,2018-06-01,0,1
6,2018-01-01,1,0
7,2018-02-01,1,1
8,2018-03-01,1,1
9,2018-04-01,1,1


**Simulating the CPS**

Just to "keep it interesting", let's tie our hands in a similar way to how the BLS has their hands tied.

We will simulate an individual's full employment history, but will only keep the subset that corresponds to when they would have been interviewed by the CPS

In [70]:
# start yearr and start month to when the first interview occurs
# cps is going to contain only information about start date and start month
# we are gong to starrt 4 months of observations in the first year
# how to interview--> 

In [71]:
def cps_interviews(df, start_year, start_month):
    """
    Takes an individual simulated employment/unemployment
    history and "interviews" the individual as if they were
    in the CPS
    
    Parameters
    ----------
    df : pd.DataFrame
        A DataFrame with at least the columns `pid`, `dt`,
        and `employment`
    start_year : int
        The year in which their interviewing begins
    start_month : int
        The month in which their interviewing begins

    Returns
    -------
    cps : pd.DataFrame
        A DataFrame with the same columns as `df` but only
        with observations that correspond to the CPS
        interview schedule for someone who starts
        interviewing in f`{start_year}/{start_month}`
    """
    # Get dates that are associated with being interviewed in
    # the CPS
    start_date_y1 = dt.datetime(start_year, start_month, 1)
    dates_y1 = pd.date_range(start_date_y1, periods=4, freq="MS")
    start_date_y2 = dt.datetime(start_year+1, start_month, 1)
    dates_y2 = pd.date_range(start_date_y2, periods=4, freq="MS")
    dates = dates_y1.append(dates_y2)

    # FROM start_date_y1 TO start_date_y2

    # Filter data that's not in the dates
    cps = df.loc[df["dt"].isin(dates), :]

    return cps

In [72]:
# how to interview?
# use the cps function 
# assign a dataframe to it
# we are randomly going to choose a year and an integeer

In [73]:
interview = lambda x: cps_interviews(
    x,
    np.random.choice(x["dt"].dt.year.unique()),
    np.random.randint(1, 13)
)

# then we are going to group by the person's id

cps_data = (
    df.groupby("pid")
      .apply(
          lambda x: interview(x)
      )
      .reset_index(drop=True)
)

In [74]:
cps_data.head(25)

# we stop the interviwes on january 2020 so this person's interviews werre cut short
# this is the same person and we are looking at his states

,dt,pid,employment
0,2018-04-01,1,1
1,2018-05-01,1,1
2,2018-06-01,1,1
3,2018-04-01,3,0
4,2018-05-01,3,0
5,2018-06-01,3,0
6,2018-02-01,4,1
7,2018-03-01,4,1
8,2018-04-01,4,1
9,2018-05-01,4,1


**How many people are we observing per month?**

If we think about the pattern used for the CPS interviews, we can form an idea of how many people might be interviewed each month.

Consider if we started interviewing $m$ new individuals per month. How many would we be interviewing in any given month?

Well. We'd at least be interviewing the $m$ new individuals. We would also be interviewing all of the individuals that had started their interviews in the previous 3 months. Additionally, we would be interviewing all of the individuals who had begun their interviews during those four months of the previous year.

We can see this below -- Note that our "survey" begins in January 2018, so at first we only have $m$ individuals being interviewed, but, as the survey progresses, we move towards $8 m$ individuals being interviewed each month.

In [75]:
cps_data["dt"].value_counts().sort_index()

# we previously interviewd 5000 individuals overall
# we are going to add about 200 people per month
# all of this is good because i get 800 from the 4th month


2018-01-01     381
2018-02-01     807
2018-03-01    1230
2018-04-01    1647
2018-05-01    1681
2018-06-01    1673
Name: dt, dtype: int64

In [76]:
# our data looks like 

**Fitting to a cross-section**

Well... Now our data look exactly like what the BLS uses to estimate the EU and UE transition rates from the raw data.

In order to fit the data, we are going to continue using the "frequency of transition" concept that we previously proposed, but, we must account for the shape of the data we receive now.

Let's think about the general case. Consider an $N$-state Markov chain. The parameters of the Markov chain are the elements of the transition matrix, $P$.

$$P \equiv \begin{bmatrix} p_{11} & p_{12} & \dots & p_{1N} \\ p_{21} & \vdots & \ddots & \vdots \\ p_{N1} & p_{N2} & \dots & p_{NN} \end{bmatrix}$$

Let $\{ \{y_{i, 0}, y_{i, 1}, \dots, y_{i, T_i}\} \; \forall i \in \{0, 1, \dots, I\}\}$ be a $I$ sequences of observations generated from the $N$-state Markov chain, then our new "fitting" procedure would assign the following value to $p_{ij}$:

$$p_{ij} = \frac{\sum_{m=0}^I \sum_{t=0}^T \mathbb{1}_{y_{m, t} == i} \mathbb{1}_{y_{m, t+1} == j}}{\sum_{m=0}^I \sum_{t=0}^T \mathbb{1}_{y_{m, t} == i}}$$


In [77]:
# now our data will look y00, y01...
# we are simply going to add a sum

**Cross-sectional counting frequencies**

In [78]:
def cps_count_frequencies(df):
    """
    Estimates the transition probability from employment
    and unemployment histories of a CPS sample of
    individuals
    
    Parameters
    ----------
    df : pd.DataFrame
        A sample of individuals from the CPS survey. Must
        have columns `dt`, `pid`, and `employment`.

    Returns
    -------
    alpha : float
        The probability of transitioning from unemployment
        to employment
    beta : float
        The probability of transitioning from employment
        to unemployment
    """
    # Set the index to be dt/pid
    data_t = df.set_index(["dt", "pid"])

    # Now find the "t+1" months and "pid"s
    tp1 = data_t.index.get_level_values("dt").shift(periods=1, freq="MS") # we are going to shift the data forward by one month
    pid = data_t.index.get_level_values("pid") # we are going to extract all the persons associated ID
    idx = pd.MultiIndex.from_arrays([tp1, pid], names=["dt", "pid"])

    # Now "index" into the data and reset index
    data_tp1 = (
        data_t.reindex(idx)
            .rename(columns={"employment": "employment_tp1"})
    )
    # we are going to create the new index with the new data

    # now we arer coing to concaat the data and only keep ["dt", "pid", "employment"] and ["employment_tp1"]
    # it means that we are resetting the index in both data frames: dt and pid will become columns
    # new 
    out = pd.concat(
        [
            data_t.reset_index().loc[:, ["dt", "pid", "employment"]],
            data_tp1.reset_index()["employment_tp1"]
        ], axis=1, sort=True
    ).dropna(subset=["employment_tp1"])
    out["employment_tp1"] = out["employment_tp1"].astype(int)


    # NOW THAT WE HAVE THE COLUMNS OF THE NEW DATAFRAME WE CAN COUNT THE FREQUENCY OF PASSING FROM ONE STATE TO THE OTHER
    # Count how frequently we go from 0 to 1
    out_zeros = out.query("employment == 0")
    alpha = out_zeros["employment_tp1"].mean()
    
    # Count how frequently we go from 1 to 0
    out_ones = out.query("employment == 1")
    beta = (1 - out_ones["employment_tp1"]).mean()

    return alpha, beta

**Checking accuracy**

In [79]:
def check_accuracy_cs(N, T, alpha=0.25, beta=0.025):
    """
    Checks the accuracy of our fit by printing the true values
    and the fitted values for a given T
    
    Parameters
    ----------
    N : int
        The total number of people we ever interview
    T : int
        The length of our simulation
    alpha : float
        The probability that an individual goes from
        unemployed to employed
    beta : float
        The probability that an individual goes from
        employed to unemployed
    """
    alpha_beta_df = pd.DataFrame(
        {
            "dt": pd.date_range("2018-01-01", periods=T, freq="MS"), 
            "alpha": np.ones(T)*alpha,
            "beta": np.ones(T)*beta
        }
    )

    # Simulate the full cross-section
    frac_unemployed = beta / (alpha + beta)
    frac_employed = alpha / (alpha + beta)
    df = pandas_employment_cross_section(
        alpha_beta_df, np.array([frac_unemployed, frac_employed]), N
    )

    # Interview individuals according to the cps interviews
    interview = lambda x: cps_interviews(
        x,
        np.random.choice(df["dt"].dt.year.unique()),
        np.random.randint(1, 13)
    )
    cps_data = (
        df.groupby("pid")
          .apply(
              lambda x: interview(x)
          )
          .reset_index(drop=True)
    )

    # Check the fit
    alpha_hat, beta_hat = cps_count_frequencies(cps_data)
    
    print(f"True alpha was {alpha} and fitted value was {alpha_hat}")
    print(f"True beta was {beta} and fitted value was {beta_hat}")
    
    return alpha, alpha_hat, beta, beta_hat

In [80]:
check_accuracy_cs(1_000, 24, 0.25, 0.025)

True alpha was 0.25 and fitted value was 0.21866666666666668
True beta was 0.025 and fitted value was 0.028642247314789315


(0.25, 0.21866666666666668, 0.025, 0.028642247314789315)

In [81]:
check_accuracy_cs(500, 24, 0.25, 0.025)

True alpha was 0.25 and fitted value was 0.22033898305084745
True beta was 0.025 and fitted value was 0.028384279475982533


(0.25, 0.22033898305084745, 0.025, 0.028384279475982533)

In [82]:
check_accuracy_cs(100, 24, 0.25, 0.025)

# here it gets pretty noisy because the sample is very small

True alpha was 0.25 and fitted value was 0.3076923076923077
True beta was 0.025 and fitted value was 0.0243161094224924


(0.25, 0.3076923076923077, 0.025, 0.0243161094224924)

## Step 7: Fit the model with actual CPS data

We've downloaded (and cleaned!) a subset of real CPS data for the years 2018 and 2019.

Let's see what our constant parameter model does with this data.

In [83]:
pip install pyarrow

You should consider upgrading via the '/Users/fabrizio/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [84]:
# Load real CPS data that we've cleaned for you.
cps_data = pd.read_parquet("cps_data.parquet")

**What does this data contain?**

In [85]:
cps_data.head()

,dt,pid,employment
0,2018-01-01,20161200000201,1
1,2018-01-01,20180100000301,1
2,2018-01-01,20180100000302,1
3,2018-01-01,20171000000401,1
4,2018-01-01,20171000000404,1


**Finding an employment history**

In [86]:
# we are going to groupby the person id
# we are going to coiunt how many times a dt shows up

cps_count_sum = cps_data.groupby("pid").agg(
    {"dt": "count", "employment": "sum"}
).sort_values("dt")

cps_count_sum.tail()


,dt,employment
pid,,
20180400009801,8,8
20180400005301,8,8
20180400004902,8,8
20180307216102,8,8
20180602828701,8,8


In [87]:
cps_data.query("pid == 20180602828701")

,dt,pid,employment
331792,2018-06-01,20180602828701,1
393339,2018-07-01,20180602828701,1
454315,2018-08-01,20180602828701,1
515616,2018-09-01,20180602828701,1
1055175,2019-06-01,20180602828701,1
1114222,2019-07-01,20180602828701,1
1173415,2019-08-01,20180602828701,1
1232918,2019-09-01,20180602828701,1


**Find an individual who experiences unemployment?**

In [88]:
cps_count_sum.query("(dt == 8) & (employment < 8)")

,dt,employment
pid,,
20180700544702,8,7
20180700538502,8,5
20180700436901,8,7
20180700434903,8,7
20180700693902,8,7
...,...,...
20180307173302,8,6
20180307157201,8,4
20180307145102,8,7


In [89]:
cps_data.query("pid == 20180307173302")

,dt,pid,employment
183418,2018-03-01,20180307173302,1
245206,2018-04-01,20180307173302,1
306813,2018-05-01,20180307173302,1
368397,2018-06-01,20180307173302,1
913110,2019-03-01,20180307173302,1
972508,2019-04-01,20180307173302,0
1031130,2019-05-01,20180307173302,1
1090177,2019-06-01,20180307173302,0


**Computing $\alpha$ and $\beta$**

In [90]:
alpha_cps, beta_cps = cps_count_frequencies(cps_data)

In [91]:
alpha_cps

0.3722666870907211

In [92]:
beta_cps

0.009071803038838207

In [93]:
beta_cps / (alpha_cps + beta_cps)

0.023789371578400262